## Imports 

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import Data_Explo_HELPER as halp
import Compl_Algo_Helper as comp_halp

## Grid_Search

In [2]:
#Set patient
patient = 11

#load data
breathD,esoData,FFFT = halp.patientManager(patient,'entire')

#Perform bucketing
esoData_bins = comp_halp.bin_divider(esoData,breathD)


hk_parts = comp_halp.grid_search(esoData, esoData_bins,breathD,patient,verbose=True)

created DF
NOTE: For patient 5, 16 and 21, missing values have been imputated 
The imputated values are are:
Patient 5: part 3 
Patient 16: part 4 
Patient 21: part 2
****CURRENT PATIENT IS: 11****
**CURRENT PART IS: 1**
breath 0 j = 1.4362434565656774
breath 90 j = 31.071379939586368
total loss for part 1 is: 811767.9513162867
**CURRENT PART IS: 2**
breath 0 j = 15.785378931388559
breath 90 j = 36.47703892124446
total loss for part 2 is: 1062092.7062781795
**CURRENT PART IS: 3**
breath 0 j = 0.014856184537956747
breath 90 j = 24.366290273863562
total loss for part 3 is: 815861.1225147815
**CURRENT PART IS: 4**
breath 0 j = 7.503731939618104
breath 90 j = 45.2259550977166
total loss for part 4 is: 1188997.4153524686
**CURRENT PART IS: 5**
breath 0 j = 3.101455597540715
breath 90 j = 27.707135392931644
total loss for part 5 is: 1083800.2888395377
**CURRENT PART IS: 6**
breath 0 j = 7.871469529659205
breath 90 j = 94.12972903402022
total loss for part 6 is: 3083723.460301426
**CURRENT PA

In [3]:
hk_parts

{1: {'tot_loss': 811767.9513162867,
  'j_best': 2.044444524719377e-10,
  'C_best': 278.0,
  'b_best': 27},
 2: {'tot_loss': 1062092.7062781795,
  'j_best': 2.4144747331879567e-08,
  'C_best': 157.0,
  'b_best': 7},
 3: {'tot_loss': 815861.1225147815,
  'j_best': 2.551759878780339e-09,
  'C_best': 363.5,
  'b_best': 118},
 4: {'tot_loss': 1188997.4153524686,
  'j_best': 7.029811208382419e-08,
  'C_best': 219.5,
  'b_best': 123},
 5: {'tot_loss': 1083800.2888395377,
  'j_best': 1.0644209904164863e-07,
  'C_best': 353.0,
  'b_best': 38},
 6: {'tot_loss': 3083723.460301426,
  'j_best': 6.906911180925095e-11,
  'C_best': 397.0,
  'b_best': 73},
 7: {'tot_loss': 3600983.743827616,
  'j_best': 3.3322360851477546e-07,
  'C_best': 362.5,
  'b_best': 80},
 8: {'tot_loss': 2772791.352603105,
  'j_best': 1.6415215143216858e-08,
  'C_best': 350.5,
  'b_best': 102}}

## Plot Pmus at best Compliance vs. actual measurements

### Calculate pmus

In [4]:
# Set part
part = 6

# Create proper colnames
prefix = str(part)
ModifiedPeso = prefix + 'ModifiedPeso'
ModifiedPao = prefix + 'ModifiedPao'
breath_col = prefix + 'breath'
Vt = prefix + 'Vt'

#Get PEEP
PS, PEEP = halp.Ventilator().get_Settings(11)
PEEP = PEEP[part]

#Set compliance from best estimated compliance
C = hk_parts[part]['C_best']

#Set breath amount, by checking length of the column without NaN's
b_amt = len(esoData_bins[ModifiedPao].dropna())

#Create DF for storing calc values
calc_peso = pd.DataFrame(index=np.arange(b_amt),columns=['breath_col','calc_peso'])



for breath in range(b_amt):    
    calc_peso.at[breath,'calc_peso'] = ((breathD.at[breath,Vt]*100)/C) + PEEP - esoData_bins.at[breath,ModifiedPao]
    calc_peso.at[breath,'breath'] = breath
    

NOTE: For patient 5, 16 and 21, missing values have been imputated 
The imputated values are are:
Patient 5: part 3 
Patient 16: part 4 
Patient 21: part 2


In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode



trace1 = go.Scatter(
    x=esoData_bins[breath_col],
    y=esoData_bins[ModifiedPeso],
    name=ModifiedPeso,
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=calc_peso['breath'],
    y=calc_peso['calc_peso'],
    name='Calculated Peso',
    yaxis='y2'

)

trace3 = go.Scatter(
    x=calc_peso['breath'],
    y=calc_peso['calc_peso'],
    name='Calculated Peso'
)

fig = make_subplots(rows=2,cols=1,specs=[[{"secondary_y": True}],[{'rowspan' : 1}]]) #,
fig.add_trace(trace1,row=1,col=1)
fig.add_trace(trace2,row=1,col=1,secondary_y=True) #secondary_y=True,
fig.add_trace(trace3,row=2,col=1)
fig['layout'].update(height = 600, width = 800, title = "Peso vs. Estimated Peso",xaxis=dict(
      tickangle=-90
    ))
fig.show()

In [18]:
fig.print_grid

<bound method BaseFigure.print_grid of Figure({
    'data': [{'marker': {'color': 'rgb(34,163,192)'},
              'name': '6ModifiedPeso',
              'type': 'scatter',
              'x': array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
                           12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
                           24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,
                           36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
                           48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
                           60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
                           72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,
                           84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,
                           96.,  97.,  98.